In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import re
import json
import os

In [2]:
# with open(file='file.json', mode='r', encoding='utf-8') as reader:
#     store_list = json.load(reader)
store_list = [{"name": "中坡", "address": "台北市南港區中坡南路47號1樓"},
              {"name": "中研", "address": "台北市南港區研究院路二段128號1樓(學術活動中心)"},
              {"name": "中貿", "address": "台北市南港區經貿二路186號2樓"},
              {"name": "玉成", "address": "台北市南港區西新里南港路三段3號1樓"}]

In [3]:
if os.path.exists('adjacency_matrix.json'):
    print('open file')
    with open(file='adjacency_matrix.json', mode='r', encoding='utf-8') as reader:
        adjacency_matrix = json.load(reader)
else:
    print('create file')
    # option = input('Y/N')
    # if option != 'Y':
    #     exit()
    adjacency_matrix = {}
    for i in store_list:
        adjacency_matrix[i['name']] = {}
        for j in store_list:
            adjacency_matrix[i['name']][j['name']] = 0
    with open(file='adjacency_matrix.json', mode='w', encoding='utf-8') as writer:
        json.dump(obj=adjacency_matrix, fp=writer, indent=4)

pd.options.display.max_columns = None
pd.options.display.max_rows = None
# pd.options.display.max_columns = 10
# pd.options.display.max_rows = 10
pd.DataFrame.from_dict(adjacency_matrix)

create file


,中坡,中研,中貿,玉成
中坡,0,0,0,0
中研,0,0,0,0
中貿,0,0,0,0
玉成,0,0,0,0


In [4]:
template = 'https://www.google.com.tw/maps/dir/{}/{}'

In [5]:
def get_dist(soup):
    doc = soup.prettify()
#     pos_list = [m.start() for m in re.finditer('公里', doc)]
#     if pos_list == []:
#         pos_list = [m.start() for m in re.finditer('公尺', doc)]
#     for i in pos_list:
#         print(doc[i-20:i+2])

    if doc.find('公里') != -1:
        pos = doc.find('公里')
        return float(doc[pos-20:pos].split('\"')[-1])

    elif doc.find('公尺') != -1:
        pos = doc.find('公尺')
        return float(doc[pos-20:pos].split('\"')[-1]) / 1000
    else:
        return -1

In [6]:
flag = False
for index, i in enumerate(store_list):
    if flag:
      break
    for j in store_list[index+1:]:
        print('{} <-> {}'.format(i['name'], j['name']))
        if adjacency_matrix[i['name']][j['name']] != 0:
            continue
        url = template.format(i['address'], j['address'])
        print(url)
        html_doc = requests.get(url)
        if html_doc.status_code != 200:
          flag = True
          print('I\'m a robot')
          break
        # assert html_doc.status_code == 200
        soup = BeautifulSoup(html_doc.text, 'html.parser')
        dist = get_dist(soup)
        if dist <= 0:
          flag = True
          print('I\'m a robot')
          break
        # assert dist > 0
        adjacency_matrix[i['name']][j['name']] = dist
        adjacency_matrix[j['name']][i['name']] = dist
        print('{} km'.format(dist))
        print('-'*50)
        sleep(1)

中坡 <-> 中研
https://www.google.com.tw/maps/dir/台北市南港區中坡南路47號1樓/台北市南港區研究院路二段128號1樓(學術活動中心)
5.5 km
--------------------------------------------------
中坡 <-> 中貿
https://www.google.com.tw/maps/dir/台北市南港區中坡南路47號1樓/台北市南港區經貿二路186號2樓
5.9 km
--------------------------------------------------
中坡 <-> 玉成
https://www.google.com.tw/maps/dir/台北市南港區中坡南路47號1樓/台北市南港區西新里南港路三段3號1樓
2.2 km
--------------------------------------------------
中研 <-> 中貿
https://www.google.com.tw/maps/dir/台北市南港區研究院路二段128號1樓(學術活動中心)/台北市南港區經貿二路186號2樓
3.0 km
--------------------------------------------------
中研 <-> 玉成
https://www.google.com.tw/maps/dir/台北市南港區研究院路二段128號1樓(學術活動中心)/台北市南港區西新里南港路三段3號1樓
4.6 km
--------------------------------------------------
中貿 <-> 玉成
https://www.google.com.tw/maps/dir/台北市南港區經貿二路186號2樓/台北市南港區西新里南港路三段3號1樓
3.1 km
--------------------------------------------------


In [7]:
pd.DataFrame.from_dict(adjacency_matrix)

,中坡,中研,中貿,玉成
中坡,0.0,5.5,5.9,2.2
中研,5.5,0.0,3.0,4.6
中貿,5.9,3.0,0.0,3.1
玉成,2.2,4.6,3.1,0.0


In [8]:
with open(file='adjacency_matrix.json', mode='w', encoding='utf-8') as writer:
    json.dump(obj=adjacency_matrix, fp=writer, indent=4)